# Neural Arousal Dynamics Exploration

Interactive exploration of sexual arousal mechanisms and computational models.

**Learning Objectives:**
1. Understand the dual control model of arousal
2. Explore how excitation and inhibition interact
3. Investigate individual differences through parameter variation
4. Visualize arousal dynamics in different contexts

**Prerequisites:**
- Basic Python knowledge
- Understanding of differential equations (helpful but not required)
- Familiarity with arousal neuroscience (see README.md)

## Setup

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add scripts to path
sys.path.append('../scripts')

from models.arousal_dynamics import ArousalModel, example_stimuli

# Set plotting style
sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)
%matplotlib inline

print("Setup complete!")

## Part 1: Basic Arousal Model

Let's start with a simple arousal model and see how it responds to constant stimulation.

### The Dual Control Model

The model implements two competing processes:

**Excitation (E):**
- Driven by sexual stimuli
- Increases arousal
- Can amplify itself (positive feedback)

**Inhibition (I):**
- Built up by excitation
- Decreases arousal
- Provides regulatory control

**Net Arousal = E - I**

In [ ]:
# Create model with default parameters
model = ArousalModel()

# Simulate constant stimulus
t, E, I, arousal = model.simulate(duration=50)

# Plot
fig = model.plot(t, E, I, arousal)
plt.show()

print(f"\nPeak arousal: {arousal.max():.2f}")
print(f"Time to peak: {t[arousal.argmax()]:.1f} time units")

### Observations

Notice:
1. **Excitation (green)** rises quickly when stimulus is applied
2. **Inhibition (red)** builds up more slowly
3. **Net arousal (blue)** peaks and then decreases as inhibition catches up
4. System reaches equilibrium where E and I balance

This demonstrates how arousal is self-regulating.

## Part 2: Different Stimulus Patterns

How does arousal respond to different types of stimulation?

Let's test:
- **Pulse**: Brief stimulation
- **Ramp**: Gradually increasing then decreasing
- **Intermittent**: On-off pattern

In [ ]:
# Define different stimuli
def pulse(t):
    """Brief pulse of stimulation"""
    return 1.0 if 10 < t < 30 else 0.0

def ramp(t):
    """Gradual increase then decrease"""
    return min(t / 20, 1.0) if t < 40 else max(1.0 - (t - 40) / 20, 0.0)

def intermittent(t):
    """On-off pattern"""
    return 1.0 if int(t / 5) % 2 == 0 else 0.0

stimuli = {
    'Pulse': pulse,
    'Ramp': ramp,
    'Intermittent': intermittent
}

# Plot comparison
fig, axes = plt.subplots(len(stimuli), 1, figsize=(12, 10), sharex=True)

for idx, (name, stim_func) in enumerate(stimuli.items()):
    t, E, I, arousal = model.simulate(duration=60, stimulus=stim_func)
    
    axes[idx].plot(t, arousal, linewidth=2.5, label='Arousal', color='blue')
    axes[idx].plot(t, [stim_func(ti) for ti in t], '--', alpha=0.6, 
                   linewidth=1.5, label='Stimulus', color='orange')
    axes[idx].set_ylabel('Response', fontsize=11)
    axes[idx].set_title(f'{name} Stimulus', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].axhline(y=0, color='k', linestyle=':', alpha=0.3)

axes[-1].set_xlabel('Time (arbitrary units)', fontsize=12)
plt.tight_layout()
plt.show()

### Question for Reflection

Which stimulus pattern produces:
1. The highest peak arousal?
2. The most sustained arousal?
3. The most variability?

Think about real-world implications...

## Part 3: Individual Differences

People differ in their arousal responses. The model captures this through parameter variation.

### Excitation Rate (α)
How quickly someone becomes aroused in response to stimuli.

In [ ]:
# Vary excitation rate
excitation_rates = [0.5, 1.0, 1.5, 2.0]

plt.figure(figsize=(12, 6))

for rate in excitation_rates:
    m = ArousalModel(excitation_rate=rate)
    t, E, I, arousal = m.simulate(duration=50)
    plt.plot(t, arousal, linewidth=2.5, label=f'α = {rate}')

plt.xlabel('Time', fontsize=12)
plt.ylabel('Arousal', fontsize=12)
plt.title('Effect of Excitation Rate (Individual Differences in Responsiveness)', 
          fontsize=13, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\nInterpretation:")
print("- Higher α = More responsive to stimulation")
print("- Lower α = Requires stronger/longer stimulation")

### Inhibition Rate (β)
How quickly inhibitory processes build up.

In [ ]:
# Vary inhibition rate
inhibition_rates = [0.2, 0.5, 0.8, 1.1]

plt.figure(figsize=(12, 6))

for rate in inhibition_rates:
    m = ArousalModel(inhibition_rate=rate)
    t, E, I, arousal = m.simulate(duration=50)
    plt.plot(t, arousal, linewidth=2.5, label=f'β = {rate}')

plt.xlabel('Time', fontsize=12)
plt.ylabel('Arousal', fontsize=12)
plt.title('Effect of Inhibition Rate (Individual Differences in Inhibition)', 
          fontsize=13, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\nInterpretation:")
print("- Higher β = Stronger inhibition, lower peak arousal")
print("- Lower β = Weaker inhibition, higher sustained arousal")
print("- Relates to anxiety, stress, mood effects")

## Part 4: Phase Space Analysis

Visualize the dynamical system in Excitation-Inhibition space.

This shows how E and I change relative to each other over time.

In [ ]:
# Run simulation
t, E, I, arousal = model.simulate(duration=100)

# Plot phase space
fig = model.plot_phase_space(E, I)
plt.show()

print("\nPhase Space Interpretation:")
print("- Trajectory shows how E and I evolve together")
print("- System spirals toward equilibrium")
print("- Equilibrium = balance between excitation and inhibition")

## Part 5: Custom Scenarios

Create your own stimulus patterns and explore!

### Example: Simulating "Edging" (Repeated Arousal-Withdrawal)

In [ ]:
def edging_stimulus(t):
    """
    Simulates repeated arousal and withdrawal.
    Three cycles of stimulation with breaks.
    """
    # Cycle 1: t=5-15
    if 5 < t < 15:
        return 1.0
    # Break: t=15-25
    elif 25 < t < 35:
        return 1.0
    # Break: t=35-45
    elif 45 < t < 65:
        return 1.0
    else:
        return 0.0

# Simulate
t, E, I, arousal = model.simulate(duration=80, stimulus=edging_stimulus)

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(t, arousal, linewidth=2.5, label='Arousal', color='blue')
ax.plot(t, [edging_stimulus(ti) for ti in t], '--', alpha=0.6, 
        linewidth=1.5, label='Stimulus', color='orange')
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Response', fontsize=12)
ax.set_title('"Edging" Pattern: Repeated Arousal-Withdrawal Cycles', 
             fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle=':', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservations:")
print(f"- Peak arousal in cycle 1: {arousal[50:150].max():.2f}")
print(f"- Peak arousal in cycle 2: {arousal[250:350].max():.2f}")
print(f"- Peak arousal in cycle 3: {arousal[450:650].max():.2f}")
print("\nNotice how inhibition builds up across cycles!")

## Part 6: Your Turn!

Experiment with the model:

1. **Create a custom stimulus function**
2. **Modify model parameters** to represent different individuals
3. **Test hypotheses** about arousal dynamics

Some ideas:
- What happens with extremely high excitation and low inhibition?
- Can you create a stimulus pattern that maintains constant arousal?
- How does varying positive feedback strength (eta) affect peak arousal?

In [ ]:
# Your experiments here!

# Example: Create your own stimulus
def my_stimulus(t):
    # Define your stimulus pattern
    # Return value between 0 and 1
    return 0.0  # Replace with your function

# Example: Create custom model
my_model = ArousalModel(
    excitation_rate=1.0,  # Adjust these
    inhibition_rate=0.5,
    coupling=0.3,
    feedback_strength=0.2
)

# Run and plot
# t, E, I, arousal = my_model.simulate(duration=100, stimulus=my_stimulus)
# fig = my_model.plot(t, E, I, arousal)
# plt.show()

## Part 7: Comparing Multiple Individuals

Simulate how different people might respond to the same stimulus.

In [ ]:
# Define different "individuals" with different parameters
individuals = {
    'High Excitation': ArousalModel(excitation_rate=2.0, inhibition_rate=0.3),
    'High Inhibition': ArousalModel(excitation_rate=1.0, inhibition_rate=1.2),
    'Balanced': ArousalModel(excitation_rate=1.0, inhibition_rate=0.5),
    'Low Responsiveness': ArousalModel(excitation_rate=0.4, inhibition_rate=0.6),
}

# Same stimulus for all
def standard_stimulus(t):
    return 1.0 if 10 < t < 50 else 0.0

# Plot comparison
plt.figure(figsize=(14, 7))

for name, individual in individuals.items():
    t, E, I, arousal = individual.simulate(duration=70, stimulus=standard_stimulus)
    plt.plot(t, arousal, linewidth=2.5, label=name)

# Add stimulus reference
plt.plot(t, [standard_stimulus(ti) for ti in t], 'k--', alpha=0.3, 
         linewidth=1.5, label='Stimulus')

plt.xlabel('Time', fontsize=12)
plt.ylabel('Arousal', fontsize=12)
plt.title('Individual Differences: Same Stimulus, Different Responses', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle=':', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nThis illustrates the dual control model:")
print("- Same stimulus → Very different arousal patterns")
print("- Depends on individual SES (excitation) and SIS (inhibition)")
print("- Clinical relevance for personalized treatment")

## Conclusions

This simple model demonstrates key principles:

1. **Arousal emerges from competition between excitation and inhibition**
2. **Different stimulus patterns produce different arousal trajectories**
3. **Individual differences arise from parameter variations (E/I balance)**
4. **The system is self-regulating through feedback**

### Limitations of This Model

This is a simplified model. It doesn't include:
- Specific neurotransmitters (dopamine, serotonin, etc.)
- Hormonal dynamics (testosterone, estrogen, prolactin)
- Brain region specificity
- Time delays in neural/endocrine processes
- Stochasticity (randomness)

### Next Steps

To extend the model:
1. Add hormone variables with slower dynamics
2. Incorporate time delays
3. Add noise to capture variability
4. Fit to empirical data
5. Model specific clinical dysfunctions

### Resources

- [README.md](../README.md) - Theoretical frameworks
- [Glossary](../docs/glossary.md) - Terminology
- [Temporal Phases](../diagrams/temporal_phases.md) - Time-course details
- [Feedback Loops](../diagrams/feedback_loops.md) - Regulatory mechanisms

## Exercises

Try these challenges:

**Beginner:**
1. Create a stimulus that starts weak and gradually increases
2. Find parameter values that produce no arousal despite stimulation
3. Measure how long it takes to return to baseline after stimulus ends

**Intermediate:**
1. Model the refractory period (post-orgasm arousal suppression)
2. Create a stimulus pattern that maintains arousal at exactly 0.5
3. Find the optimal stimulus pattern for maximum sustained arousal

**Advanced:**
1. Add a third variable for hormonal state
2. Implement stochastic (random) fluctuations
3. Fit the model to hypothetical experimental data
4. Model a specific sexual dysfunction and test interventions

---

**Notebook:** arousal_exploration.ipynb  
**Author:** Deep Convo GPT Research Team  
**Date:** 2025-11-19  
**Version:** 1.0  